In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
data_sample = spark.read.option("header","true").parquet("dbfs:/mnt/group12/filtered/")

In [0]:
data_sample.count()

Out[3]: 1918155

In [0]:
%pip install vaderSentiment

Python interpreter will be restarted.
Collecting vaderSentiment
 Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from vaderSentiment) (2.25.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (2.10)
Installing collected packages: vaderSentiment
Successfully installed vaderSentiment-3.3.2
WARNING: You are using pip version 21.0.1; however, version 21.3 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9a72f1d9-040e-4844-bed0-ae5b205f2f30/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
# Vader Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

def sentiment_vader(text):
  if (text == None or len(text) < 2):
    return -69
  sentiment_dict = vader.polarity_scores(text)
  if ("compound" in sentiment_dict):
    return sentiment_dict["compound"]
  return -69

sentiment_vader_udf = F.udf(lambda x: sentiment_vader(x), FloatType()) 

In [0]:
data_sample_sentiment = data_sample.withColumn("vader_sentiment", sentiment_vader_udf(F.col("body")))

In [0]:
%pip install liwc

Python interpreter will be restarted.
Collecting liwc
 Using cached liwc-0.5.0-py2.py3-none-any.whl (5.1 kB)
Installing collected packages: liwc
Successfully installed liwc-0.5.0
WARNING: You are using pip version 21.0.1; however, version 21.3 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9a72f1d9-040e-4844-bed0-ae5b205f2f30/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
# LIWC Linguistic Dimensions Analysis
import liwc
import re
from collections import Counter

parse, category_names = liwc.load_token_parser('/dbfs/mnt/group12/liwc_en2.dic')

def liwc_analysis(text):
    if (text == None or len(text) < 2):
        return {"tokens": 0}
    tokens = text.split(' ')
    liwc_magic = Counter(category for token in tokens for category in parse(token))
    liwc_results = dict(liwc_magic)
    i = len(tokens)
    for category in category_names:
        if category in liwc_results:
            print(category)
            liwc_results[category] = round(((float(liwc_results[category]) / i) * 100), 2)
    liwc_results["tokens"] = i
    return liwc_results
    
sentiment_liwc_udf = F.udf(lambda x: liwc_analysis(x), MapType(StringType(), FloatType())) 

In [0]:
data_sample_sentiment = data_sample_sentiment.withColumn("liwc_sentiment", sentiment_liwc_udf(F.col("body")))

In [0]:
# Writing output
data_sample_sentiment.write.mode('overwrite').partitionBy('created_at_month').parquet("dbfs:/mnt/group12/sentiment/")

In [0]:
# Testing output
data_output = spark.read.option("header","true").parquet("dbfs:/mnt/group12/sentiment/")

In [0]:
data_output.printSchema()

root
-- created_at_dt: date (nullable = true)
-- word_count: integer (nullable = true)
-- category_title: string (nullable = true)
-- body: string (nullable = true)
-- created_at: string (nullable = true)
-- dislike_count: long (nullable = true)
-- id: long (nullable = true)
-- like_count: long (nullable = true)
-- repost_count: long (nullable = true)
-- reply_count: long (nullable = true)
-- score: long (nullable = true)
-- topic_created_at: string (nullable = true)
-- topic_id: string (nullable = true)
-- topic_title: string (nullable = true)
-- user_id: long (nullable = true)
-- username: string (nullable = true)
-- user_real_name: string (nullable = true)
-- repost: boolean (nullable = true)
-- quote_conversation_parent: struct (nullable = true)
 |-- attachment: struct (nullable = true)
 | |-- type: string (nullable = true)
 | |-- value: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- body_html_summary: string (nullable = true)
 |-- body_html_summary_truncated: boolean (nullable = true)
 |-- bookmarked: boolean (nullable = true)
 |-- category: long (nullable = true)
 |-- category_details: struct (nullable = true)
 | |-- emoji: string (nullable = true)
 | |-- slug: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- value: long (nullable = true)
 |-- created_at: string (nullable = true)
 |-- dislike_count: long (nullable = true)
 |-- disliked: boolean (nullable = true)
 |-- edited: boolean (nullable = true)
 |-- embed: struct (nullable = true)
 | |-- html: string (nullable = true)
 | |-- iframe: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- is_locked: boolean (nullable = true)
 |-- is_premium: boolean (nullable = true)
 |-- is_quote: boolean (nullable = true)
 |-- is_replies_disabled: boolean (nullable = true)
 |-- is_reply: boolean (nullable = true)
 |-- language: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- liked: boolean (nullable = true)
 |-- nsfw: boolean (nullable = true)
 |-- only_emoji: boolean (nullable = true)
 |-- poll: struct (nullable = true)
 | |-- option_1_title: string (nullable = true)
 | |-- option_1_votes: long (nullable = true)
 | |-- option_2_title: string (nullable = true)
 | |-- option_2_votes: long (nullable = true)
 | |-- option_3_title: string (nullable = true)
 | |-- option_3_votes: long (nullable = true)
 | |-- option_4_title: string (nullable = true)
 | |-- option_4_votes: long (nullable = true)
 | |-- option_5_title: string (nullable = true)
 | |-- option_5_votes: long (nullable = true)
 | |-- total_votes: long (nullable = true)
 | |-- voted_option: string (nullable = true)
 |-- replies: struct (nullable = true)
 | |-- data: array (nullable = true)
 | | |-- element: string (containsNull = true)
 |-- reply_count: long (nullable = true)
 |-- reported: boolean (nullable = true)
 |-- repost: boolean (nullable = true)
 |-- repost_count: long (nullable = true)
 |-- revised_at: string (nullable = true)
 |-- score: long (nullable = true)
 |-- topic: struct (nullable = true)
 | |-- category: long (nullable = true)
 | |-- created_at: string (nullable = true)
 | |-- id: string (nullable = true)
 | |-- is_featured: boolean (nullable = true)
 | |-- title: string (nullable = true)
 | |-- user: struct (nullable = true)
 | | |-- id: long (nullable = true)
 | | |-- is_donor: boolean (nullable = true)
 | | |-- is_investor: boolean (nullable = true)
 | | |-- is_premium: boolean (nullable = true)
 | | |-- is_private: boolean (nullable = true)
 | | |-- is_pro: boolean (nullable = true)
 | | |-- name: string (nullable = true)
 | | |-- picture_url: string (nullable = true)
 | | |-- username: string (nullable = true)
 | | |-- verified: boolean (nullable = true)
 |-- user: struct (nullable = true)
 | |-- id: long (nullable = true)
 | |-- is_donor: boolean (nullable = true)
 | |-- is_investor: boolean (nullable = true)
 | |-- is_premium: boolean (nullable = true)
 | |-- is_private: boolean (nullable = 

In [0]:
data_output.select('liwc_sentiment').show(100, False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
liwc_sentiment |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
{tokens -> null} |
{prep -> 16.67, tokens -> null, relativ -> 16.67, space -> 16.67, function -> 16.67} |
{cogproc -> 7.14, social -> 7.14, conj -> 7.14, adj -> 7.14, relativ -> 21.43, verb -> 7.14, space -> 7.14, focuspresent -> 7.14, affiliation -> 7.14, drives -> 7.14, focuspast -> 7.14, negate -> 7.14, auxverb -> 7.14, function -> 35.71, prep -> 14.29, tokens -> null, time -> 14.29, certain -> 7.14} |
{relig -> 14.81, compare -> 3.7, posemo -> 1.85, cogproc -> 5.56, negemo -> 3.7, relativ -> 9.26, affect -> 5.56, feel -> 1.85, bio -> 5.56, percept -> 1.85, body -> 1.85, anger -> 3.7, space -> 3.7, ppron -> 1.85, see -> 1.85, affiliation -> 1.85, pronoun -> 3.7, auxverb -> 7.41, function -> 37.04, prep -> 16.67, tokens -> null, power -> 1.85, reward -> 1.85, focusfuture -> 3.7, social -> 3.7, adj -> 5.56, shehe -> 1.85, verb -> 16.67, quant -> 5.56, differ -> 1.85, article -> 9.26, focuspresent -> 11.11, focuspast -> 3.7, drives -> 11.11, ipron -> 1.85, risk -> 5.56, time -> 5.56, informal -> 3.7, certain -> 3.7, male -> 1.85, swear -> 3.7, sexual -> 3.7}|
{compare -> 8.7, negemo -> 8.7, cogproc -> 8.7, affect -> 8.7, cause -> 4.35, anger -> 4.35, we -> 4.35, ppron -> 4.35, interrog -> 4.35, affiliation -> 4.35, pronoun -> 4.35, auxverb -> 8.7, function -> 47.83, prep -> 21.74, tokens -> null, social -> 4.35, adj -> 8.7, verb -> 13.04, anx -> 4.35, quant -> 4.35, differ -> 4.35, article -> 8.7, focuspresent -> 13.04, drives -> 4.35, adverb -> 8.7} |
{posemo -> 6.25, cogproc -> 6.25, social -> 12.5, work -> 6.25, affect -> 6.25, conj -> 6.25, adj -> 6.25, relativ -> 12.5, percept -> 6.25, we -> 6.25, space -> 6.25, ppron -> 6.25, insight -> 6.25, see -> 6.25, affiliation -> 6.25, drives -> 12.5, pronoun -> 6.25, function -> 18.75, prep -> 6.25, tokens -> null, power -> 6.25, time -> 6.25} |
{auxverb -> 8.33, function -> 16.67, verb -> 8.33, tokens -> null, article -> 8.33, focuspresent -> 8.33} |
{cogproc -> 8.33, auxverb -> 16.67, function -> 50.0, verb -> 16.67, prep -> 8.33, tokens -> null, adverb -> 8.33, focuspresent -> 8.33, article -> 16.67, tentat -> 8.33} |
{cogproc -> 6.25, relativ -> 15.62, cause -> 3.12, we -> 3.12, space -> 9.38, ppron -> 3.12, number -> 3.12, affiliation -> 3.12, pronoun -> 12.5, auxverb -> 6.25, function -> 43.75, prep -> 15.62, tokens -> null, power -> 3.12, motion -> 3.12, social -> 12.5, verb -> 12.5, focuspresent -> 6.25, article -> 3.12, insight -> 3.12, drives -> 6.25, focuspast -> 9.38, ipron -> 9.38, time -> 6.25, adverb -> 6.25} |
{compare -> 5.88, posemo -> 11.76, affect -> 11.76, relativ -> 17.65, space -> 5.88, interrog -> 5.88, number -> 5.88, pronoun ->